# Infeasibility analysis on a cluster

Perform a simple analysis on the results of a UQ evaluation on a cluster using Dask.

In [ ]:
import easyvvuq as uq
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
print("Reading in campaign database.")

# Find DB
db_count = 0
for db_path in Path.cwd().glob("campaigns/*/campaign.db"):
    if db_count > 0:
        raise RuntimeError("More than one database in the campaigns dir")

    db_location = str(db_path.resolve())
    db_count += 1

# /// prefix is required before absolute path
db_location_prefixed = f"sqlite:///{db_location}"
campaign = uq.Campaign(
    db_location=db_location_prefixed, name="example_local", work_dir="campaigns"
)

samples = campaign.get_collation_result()
sample_count = samples.shape[0]
print(f"Campaign read in. Number of samples = {sample_count}")

# Drop strange multi-index of 0
samples.columns = samples.columns.droplevel(1)
samples

Reading in campaign database.
Campaign read in. Number of samples = 64



## Plot some samples

The most basic analysis.


In [ ]:
# Plot 2 vars from sample against each other
# Input epistemic uncertainty aspect against vio_constr_res
ax = sns.regplot(x=samples["aspect"], y=samples["rms_vio_constr_res"])
ax.set_xlabel("aspect")
ax.set_ylabel("rms_vio_constr_res")
ax.set_title("Variability in vio_constr_res against aspect ratio")

## Analysis

Analyse vio_constr_residuals, the violated constraint residuals.

### KDE for `vio_constr_res` from EasyVVUQ (plotted with seaborn)


In [ ]:
# Try to analyse all outputs variables: produces a linalg error
# results = campaign.analyse(qoi_cols=palph2_inputs)

# Analyse a single output variable, vio_constr_res
results = campaign.analyse(qoi_cols=["rms_vio_constr_res"])

# Get its distribution
dist = results.get_distribution(qoi="rms_vio_constr_res")

# Locations for density function to be evaluated
# (This is taken from easyvvuq's fusion tutorial)
x = np.linspace(dist.lower[0], dist.upper[0])
pdf = dist.pdf(x)

# Plot
ax = sns.lineplot(x=x, y=pdf, markers=True)
ax.set_title("Distribution for rms_vio_constr_res")
ax.set_xlabel("rms_vio_constr_res")
ax.set_ylabel("Probability density")

PDF for `vio_constr_res`. Appears right from looking at the EasyVVUQ tutorial (fusion Dask).

## Sobol indices


In [ ]:
# results.plot_moments(qoi="vio_constr_res")
# results.plot_sobols_first("vio_constr_res") # only for vecotr qois. Like constraint vectors?

fig, ax = plt.subplots()
results.plot_sobols_treemap(
    "rms_vio_constr_res", figsize=(10, 10), ax=ax, filename="sobols"
)